In [1]:
import numpy as np
import cv2
import urllib
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [10]:
import time
import os
import pandas as pd
from selenium import webdriver

In [2]:
import matplotlib.pyplot as plt
#import pyplot as plt

#### Getting the Names from Exotic Insects List

In [ ]:
# Original GBIF Exotic data in excel format. Reading excel file using pandas 
data = pd.read_excel("imagefilename.xlsx")
data.columns

In [202]:
list_species = data['Original species name']
list_species = list_species.unique()

In [331]:
len(names)

1500

In [332]:
names = []
for name in list_species[0:5]: 
    first = name.split()[0]
    second = name.split()[1]
    new_name = first + "-" + second
    names.append(new_name)

In [333]:
# (names[0]).lower()

### Trial Below to reach the search page

In [334]:
driver = webdriver.Firefox(executable_path=r"path/geckodriver-v0.23.0-win64/geckodriver.exe")

In [335]:
def get_url_of_insect(name, searchUrl): 
    
    for item in searchUrl:
        try :     
            if name in item:
                return item
        except TypeError:
            continue

In [336]:
def get_url_of_image(next_url):
    driver.get(next_url)
    html = driver.page_source
    soup = bs(html, "lxml")
    link_list =[]

    links  = soup.find_all("a", class_="btn btn-primary btn-inat btn-xs")
    #links
    for link in links:
        link_list.append(link.get('href'))
    
    #print(link_list)
    
    url_of_insect = []
    for links in link_list:
        if "/observations/" in links:
            url_of_insect.append(links)
    #print(url_of_insect)
    return url_of_insect
    

In [356]:
urls_for_images = []

for name in names:
    
    url = "https://anywebsite/taxa/search?q="+name.lower()
    driver.get(url)
    
    html = driver.page_source
    soup = bs(html, "lxml")
    links  = soup.find_all("a")
    #print(links)
    searchUrl= []
    for link in links:
        searchUrl.append(link.get('href'))
    #print(searchUrl)
    #print(name)
    different_species = []
    url_of_insect = get_url_of_insect(name, searchUrl)
    if url_of_insect is None:
        different_species.append(name)
        next_url = None
    else:
        next_url = "https://anywebsite" + url_of_insect
    #print(url_of_insect)
    #print(next_url)
    
    if next_url is None:
        url_of_insect_2 = []
    else:
           
        url_of_insect_2 = get_url_of_image(next_url)
    
    if len(url_of_insect_2) == 0:
        urls_for_images.append("NA")
    else:
        urls_for_images.append(url_of_insect_2[0])
    print(url_of_insect_2)

[]
[]
['/observations/18618656']
[]
['/observations/18819842']


In [367]:
df_urls_for_images = pd.DataFrame({"name" : names,  "observation" : urls_for_images})

In [368]:
 df_urls_for_images.head()

,name,observation
0,Abgrallaspis-cyanophylli,NA
1,Acallopais-rudis,NA
2,Acantholybas-brunneus,/observations/18618656
3,Acanthoscelides-obtectus,NA
4,Acanthuchus-trispinifer,/observations/18819842


In [369]:
urls = []
for i in range(len(df_urls_for_images['observation'])):
    if df_urls_for_images['observation'][i] != "NA":
        
        urls.append("https://inaturalist.nz"+df_urls_for_images['observation'][i])
    else:
        urls.append("NA")

In [370]:
df_urls_for_images["urls"] = urls

## PICKING UP IMAGES

In [373]:

image_links = []

for i in range(len(df_urls_for_images['urls'])):
    url = df_urls_for_images['urls'][i]
    if url != 'NA':
        driver.get(url)
        html = driver.page_source
        soup = bs(html, "lxml")
        links  = soup.find("div", class_ = "image-gallery-image").find_all("img", src=True)
        for link in links:
            imgUrl = link.get('src')
            image_links.append(imgUrl)
    else:
        image_links.append("NA")
        

In [376]:
df_urls_for_images["image_links"] = image_links
df_urls_for_images

,name,observation,urls,image_links
0,Abgrallaspis-cyanophylli,NA,NA,NA
1,Acallopais-rudis,NA,NA,NA
2,Acantholybas-brunneus,/observations/18618656,https://inaturalist.nz/observations/18618656,https://static.inaturalist.org/photos/28496503...
3,Acanthoscelides-obtectus,NA,NA,NA
4,Acanthuchus-trispinifer,/observations/18819842,https://inaturalist.nz/observations/18819842,https://static.inaturalist.org/photos/28827275...


In [3]:
# driver = webdriver.Firefox(executable_path=r"T:/projects/2018/SCION/geckodriver-v0.23.0-win64/geckodriver.exe")
# url = "https://inaturalist.nz/observations/18618656"
# driver.get(url)

In [4]:
# html = driver.page_source
# soup = bs(html, "lxml")

In [5]:
# links  = soup.find("div", class_ = "image-gallery-image").find_all("img", src=True)
# for link in links:
#     imgUrl = link.get('src')
#     print(imgUrl)

https://static.inaturalist.org/photos/28496503/large.jpeg?1543131054


In [384]:
def download_file(name, url):
    local_filename = url.split('/')[-1]
    
    local_filename = local_filename.split("?")[0]
    print(local_filename)
    print("Downloading {} ---> {}".format(url, local_filename))
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(name+'.jpeg', 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    

In [385]:
for i in range(len(df_urls_for_images['image_links'])):
    
    url = df_urls_for_images['image_links'][i]
    name = df_urls_for_images['name'][i]
    if url != 'NA':
        download_file(name, url)

large.jpeg
large.jpeg
